# Import Libraries

In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix
from sklearn.datasets import make_classification, make_circles
from sklearn.linear_model import Perceptron
from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm

# Import Datasets

In [69]:
seeds_train = pd.read_csv("Dataset/seeds_train.csv", header= None)    #169
seeds_test = pd.read_csv("Dataset/seeds_test.csv", header= None)      #42

In [70]:
# Make copy from our main datasets
seeds_train_ovr = seeds_train.copy()
seeds_test_ovr = seeds_test.copy()

## Method to get model accuracy 

In [71]:
# method to get accuracy of the model
def getAccuracy(model, x, y):
    return model.score(x,y)*100

In [72]:
# method to make data binary
def binary_y(df_train, df_test):
  for i in df_train[2].unique():
    df_train['y{}'.format(i)] = df_train[2].apply(lambda x: 1 if x == i else -1)
    df_test['y{}'.format(i)] = df_test[2].apply(lambda x: 1 if x == i else -1)

  return df_train, df_test

df_train_ovr, df_test_ovr = binary_y(seeds_train_ovr, seeds_test_ovr)

In [132]:
df_test_ovr.head(1)

,0,1,2,y1,y2,y3
0,14.84,2.221,1,1,-1,-1


# SVM

In [148]:
def Model_SVM(df_train_ovr, df_test_ovr):
    tmp = []
    for i in range(3):
        x_train = df_train_ovr.iloc[:, 0:2].to_numpy()
        y_train = df_train_ovr.iloc[:, (i+3)].to_numpy()

        x_test = df_test_ovr.iloc[:, 0:2].to_numpy()
        y_test = df_test_ovr.iloc[:, (i+3)].to_numpy()
        
        svm_model = svm.SVC(kernel='linear', decision_function_shape='ovo', probability=True)
        svm_model = svm_model.fit(x_train, y_train)
        
        tmp.append(svm_model.predict_proba(x_test))
    
    return tmp    

In [149]:
fl = Model_SVM(df_train_ovr, df_test_ovr)
ls_1 = fl[0][:,1]
ls_2 = fl[1][:,1]
ls_3 = fl[2][:,1]

In [150]:
predictions =[]
for i in zip(ls_1, ls_2, ls_3):
    x = np.argmax((i))
    predictions.append(x+1)
print(predictions) 

[1, 2, 2, 1, 1, 2, 1, 2, 3, 3, 2, 3, 1, 3, 2, 2, 2, 1, 3, 1, 3, 1, 2, 3, 2, 1, 1, 3, 2, 2, 2, 3, 3, 1, 2, 2, 2, 3, 2, 1, 2, 2]


In [151]:
actuals = seeds_test.iloc[: , 2]

In [152]:
print('\nConfusion Matrix:\n')
print(confusion_matrix(actuals, predictions))


Confusion Matrix:

[[11  2  0]
 [ 0 17  0]
 [ 1  0 11]]


# Perceptron

In [153]:
def Model_Perceptron(df_train_ovr, df_test_ovr):
    tmp = []
    for i in range(3):
        x_train = df_train_ovr.iloc[:, 0:2].to_numpy()
        y_train = df_train_ovr.iloc[:, (i+3)].to_numpy()

        x_test = df_test_ovr.iloc[:, 0:2].to_numpy()
        y_test = df_test_ovr.iloc[:, (i+3)].to_numpy()
        
        preceptron_model = Perceptron(alpha = 0.01, tol=1e-2, random_state=0)
        clf_isotonic = CalibratedClassifierCV(preceptron_model, method='isotonic')
        clf_isotonic.fit(x_train, y_train)
        
        tmp.append(clf_isotonic.predict_proba(x_test))
    
    return tmp  

In [154]:
fl_pre = Model_Perceptron(df_train_ovr, df_test_ovr)
ls_pre_1 = fl_pre[0][:,1]
ls_pre_2 = fl_pre[1][:,1]
ls_pre_3 = fl_pre[2][:,1]

In [155]:
predictions_prec =[]
for i in zip(ls_pre_1, ls_pre_2, ls_pre_3):
    x = np.argmax((i))
    predictions_prec.append(x+1)
print(predictions_prec) 

[1, 2, 1, 1, 1, 2, 1, 3, 3, 3, 2, 3, 1, 3, 2, 2, 2, 1, 3, 1, 3, 1, 3, 3, 2, 1, 1, 3, 2, 3, 1, 3, 3, 1, 2, 1, 2, 1, 3, 1, 2, 2]


In [147]:
print('\nConfusion Matrix:\n')
print(confusion_matrix(actuals, predictions_prec))


Confusion Matrix:

[[12  0  1]
 [ 2 12  3]
 [ 2  0 10]]
